This notebook uses submitted scientific names and taxon-mathed names from ITIS, looks for those names from the NatureServe species services, and returns key/value pairs for caching in the Taxonomic Information Registry and use within our systems. It is set up to loop the entire TIR for everything without NatureServe data until finished (with a safeguard on total records processed). We eventually need to deal with the issue of whether or not we need to cache this information over time or flush and rebuild our cache periodically.

In [6]:
import requests
from IPython.display import display
from bis import natureserve
from bis import tir
from bis2 import gc2
from bis2 import natureserve as natureservekeys

In [7]:
# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["commitToDB"] = True
thisRun["totalRecordsToProcess"] = 5
thisRun["totalRecordsProcessed"] = 0

numberWithoutTIRData = 1

while numberWithoutTIRData == 1 and thisRun["totalRecordsProcessed"] <= thisRun["totalRecordsToProcess"]:

    q_recordToSearch = "SELECT id, \
        registration->'scientificname' AS scientificname, \
        itis->'nameWInd' AS namewind, \
        itis->'nameWOInd' AS namewoind \
        FROM tir.tir \
        WHERE natureserve IS NULL \
        LIMIT 1"
    recordToSearch  = requests.get(thisRun["baseURL"]+"&q="+q_recordToSearch).json()
    
    numberWithoutTIRData = len(recordToSearch["features"])
    
    if numberWithoutTIRData == 1:
        tirRecord = recordToSearch["features"][0]

        thisRecord = {}
        thisRecord["id"] = tirRecord["properties"]["id"]
        thisRecord["scientificname"] = tirRecord["properties"]["scientificname"]
        thisRecord["nameWOInd"] = tirRecord["properties"]["namewoind"]
        thisRecord["nameWInd"] = tirRecord["properties"]["namewind"]

        thisRecord["tryNames"] = []
        thisRecord["tryNames"].append(thisRecord["scientificname"])
        if thisRecord["nameWInd"] is not None and thisRecord["nameWInd"] not in thisRecord["tryNames"]:
            thisRecord["tryNames"].append(thisRecord["nameWInd"])
        if thisRecord["nameWOInd"] is not None and thisRecord["nameWOInd"] not in thisRecord["tryNames"]:
            thisRecord["tryNames"].append(thisRecord["nameWOInd"])
        
        for name in thisRecord["tryNames"]:
            thisRecord["elementGlobalID"] = natureserve.queryNatureServeID(name)
            if thisRecord["elementGlobalID"] != "none":
                break

        # Run the function to query and pacage NatureServe key/value pairs
        thisRecord["natureServePairs"] = natureserve.packageNatureServePairs(natureservekeys.speciesAPI(),thisRecord["elementGlobalID"])

        # Display the record, cache results, and show query status
        display (thisRecord)
        if thisRun["commitToDB"]:
            print (tir.cacheToTIR(thisRun["baseURL"],thisRecord["id"],"natureserve",thisRecord["natureServePairs"]))
        thisRun["totalRecordsProcessed"] = thisRun["totalRecordsProcessed"] + 1


{'elementGlobalID': 'ELEMENT_GLOBAL.2.100745',
 'id': 20993,
 'nameWInd': 'Ammocrypta pellucida',
 'nameWOInd': 'Ammocrypta pellucida',
 'natureServePairs': '"cacheDate"=>"2017-06-28T16:49:10.279026","elementGlobalID"=>"ELEMENT_GLOBAL.2.100745","GlobalStatusRank"=>"G4","roundedGlobalStatusRankDescription"=>"Apparently Secure","globalStatusLastReviewed"=>"2011-10-19","usNationalStatusRankCode"=>"N4","usNationalStatusRoundedRankDescription"=>"Apparently Secure","usNationalStatusLastReviewed"=>"2009-08-30","StateCode:Illinois"=>"S1","StateStatus:Illinois"=>"Critically Imperiled","StateCode:Indiana"=>"S4","StateStatus:Indiana"=>"Apparently Secure","StateCode:Kentucky"=>"S4","StateStatus:Kentucky"=>"Apparently Secure","StateCode:Michigan"=>"S1S2","StateStatus:Michigan"=>"Critically Imperiled","StateCode:New York"=>"S2","StateStatus:New York"=>"Imperiled","StateCode:Ohio"=>"S3","StateStatus:Ohio"=>"Vulnerable","StateCode:Pennsylvania"=>"S1","StateStatus:Pennsylvania"=>"Critically Imperiled",

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.068, 'affected_rows': 1}


{'elementGlobalID': 'ELEMENT_GLOBAL.2.106359',
 'id': 35808,
 'nameWInd': 'Spilogale putorius interrupta',
 'nameWOInd': 'Spilogale putorius interrupta',
 'natureServePairs': '"cacheDate"=>"2017-06-28T16:49:16.162741","elementGlobalID"=>"ELEMENT_GLOBAL.2.106359","GlobalStatusRank"=>"G4T4","roundedGlobalStatusRankDescription"=>"Apparently Secure","globalStatusLastReviewed"=>"1996-11-18","usNationalStatusRankCode"=>"N4","usNationalStatusRoundedRankDescription"=>"Apparently Secure","usNationalStatusLastReviewed"=>"1997-01-15","StateCode:Arkansas"=>"S2?","StateStatus:Arkansas"=>"Imperiled","StateCode:Colorado"=>"S2","StateStatus:Colorado"=>"Imperiled","StateCode:Minnesota"=>"S2","StateStatus:Minnesota"=>"Imperiled","StateCode:Missouri"=>"S1","StateStatus:Missouri"=>"Critically Imperiled"',
 'scientificname': 'Spilogale putorius interrupta',
 'tryNames': ['Spilogale putorius interrupta']}

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.068, 'affected_rows': 1}


{'elementGlobalID': 'ELEMENT_GLOBAL.2.131565',
 'id': 30164,
 'nameWInd': 'Microseris howellii',
 'nameWOInd': 'Microseris howellii',
 'natureServePairs': '"cacheDate"=>"2017-06-28T16:49:21.096402","elementGlobalID"=>"ELEMENT_GLOBAL.2.131565","GlobalStatusRank"=>"G3","roundedGlobalStatusRankDescription"=>"Vulnerable","globalStatusLastReviewed"=>"1997-11-18","usNationalStatusRankCode"=>"N3","usNationalStatusRoundedRankDescription"=>"Vulnerable","usNationalStatusLastReviewed"=>"Unknown","StateCode:Oregon"=>"S3","StateStatus:Oregon"=>"Vulnerable"',
 'scientificname': 'Microseris howellii',
 'tryNames': ['Microseris howellii']}

...

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.066, 'affected_rows': 1}


{'elementGlobalID': 'ELEMENT_GLOBAL.2.108859',
 'id': 31199,
 'nameWInd': 'Oligia bridghamii',
 'nameWOInd': 'Oligia bridghamii',
 'natureServePairs': '"cacheDate"=>"2017-06-28T16:49:29.171758","elementGlobalID"=>"ELEMENT_GLOBAL.2.108859","GlobalStatusRank"=>"G4","roundedGlobalStatusRankDescription"=>"Apparently Secure","globalStatusLastReviewed"=>"2002-05-31","usNationalStatusLastReviewed"=>"Unknown","StateCode:Indiana"=>"S1","StateStatus:Indiana"=>"Critically Imperiled"',
 'scientificname': 'Oligia bridghami',
 'tryNames': ['Oligia bridghami', 'Oligia bridghamii']}

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.066, 'affected_rows': 1}


{'elementGlobalID': 'ELEMENT_GLOBAL.2.816980',
 'id': 22113,
 'nameWInd': None,
 'nameWOInd': None,
 'natureServePairs': '"cacheDate"=>"2017-06-28T16:49:33.549576","elementGlobalID"=>"ELEMENT_GLOBAL.2.816980","status"=>"error"',
 'scientificname': 'Berula erecta var. incisum',
 'tryNames': ['Berula erecta var. incisum']}

{'success': True, 'auth_check': {'success': True, 'auth_level': None, 'session': None}, '_execution_time': 0.065, 'affected_rows': 1}
